## Extract conservation scores for each position

### Requirements:
1. Phylop and phastCons scores arranged in "chunks". 
3. Domains list
4. Domain dictionary files

### Instructions:
Run the cells in order

### Output:
1. Creates a new dictionary for each domain with phastCons and phyloP conservation scores at each site
2. A text file with missing sites

In [1]:
import pandas as pd
import numpy as np
import cPickle as pickle
import datetime
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [2]:
#Getting path
curr_dir = !pwd
pfam_version = "32"
intance_cutoff = "10"

update_same_file = False

if (update_same_file):
    input_path = curr_dir[0]+"/ext_features_dicts/pfam-v"+pfam_version+"/"
else:
    input_path = curr_dir[0]+"/../5.HMM_alter_align/domains_states_dicts/pfam-v"+pfam_version+"/"


#Reading the list of filtered domains
if (pfam_version == "32"):
    with open(curr_dir[0]+"/../5.domains_stats/pfam-v"+pfam_version+"/regular_human_domains_list.pik", 'rb') as handle:
        filtered_domains_list = pickle.load(handle)
else:
    with open(curr_dir[0]+"/../5.domains_stats/pfam-v"+pfam_version+"/filtered"+intance_cutoff+"_list.pik", 'rb') as handle:
        filtered_domains_list = pickle.load(handle)
filtered_domains_list = filtered_domains_list.sort()

#### Helper functions - Find the file fragment corresponding to a given position on a chromosome

In [3]:
# Modified binary search to find correct fragment file
# Note: a position in a gap with still return the previous fragment — need to check if position is out of bounds later
#
# cons: "phyloP" or "phastCons"
# chrom: chromosome number
# pos: position number
def bin_search(cons,chrom,pos):
    input_path = curr_dir[0]+"/../conservation_scores/"+cons+"_frags_txt/chr"+str(chrom)+"/"
    with open(input_path+"index.pik",'rb') as handle:
        index = pickle.load(handle)
    if pos < index[0]:
        return(-1)
    return(rec_helper(0,len(index)-1,index,pos))

def rec_helper(lo,hi,a,val):
    if hi < lo:
        return(-1)
    # Indices will never be large, so computing mid in this way is fine
    mid = (hi+lo) / 2
    a_val = a[mid]
    if val >= a_val and mid == len(a)-1:
        return(a_val)
    elif val >= a_val and val < a[mid+1]:
        return(a_val)
    elif val > a_val:
        return(rec_helper(mid+1,hi,a,val))
    else:
        return(rec_helper(lo,mid-1,a,val))

### Add conservation scores to domain dictionaries

In [41]:
filtered_domains_list.index("VGLL4")

6092

In [42]:
%%time
import linecache

# Get current date
today = datetime.date.today()

# Find missing scores and write to file
missing = open(curr_dir[0]+"/../conservation_scores/missing.txt",'w')

# Loop over domains
#for i in range(0,len(filtered_domains_list)):
#for domain_name in filtered_domains_list:
for i in range(6092,len(filtered_domains_list)):
    domain_name = filtered_domains_list[i]
        
    # Reading the domain states dictionary
    domain_dirfiles = !ls -t $input_path$domain_name
    # Find the most recent file
    recent_priority = -1
    recent_filename = ""
    for f in domain_dirfiles:
        tokens = f.split("_")
        date = tokens[len(tokens)-1].split(".")
        month = int(date[0])
        day = int(date[1])
        # Not all files have years, but those that do are the most recent
        if date[2] != "pik":
            year = int(date[2])
        else:
            year = 0
        priority = year*1000 + month*50 + day
        if priority > recent_priority:
            recent_priority = priority
            recent_filename = f
    with open(input_path+domain_name+"/"+recent_filename, 'rb') as handle:
        states_dict = pickle.load(handle)

    # Loop over dictionary
    for state in states_dict:
        for d in states_dict[state]:
            for cons in ["phyloP","phastCons"]:
                # Find correct file
                frag_path = curr_dir[0]+"/../conservation_scores/"+cons+"_frags_txt/chr"+str(d['chrom'])+"/"
                # Get appropriate scores
                scores = []
                for pos in d['chrom_pos']:
                    start = bin_search(cons,str(d['chrom']),pos)
                    score = linecache.getline(frag_path+str(start)+".txt",pos-start+1).strip()
                    
                    if score != "":
                        scores.append(float(score))
                d[cons] = scores
                if len(scores) < 3:
                    missing.write(domain_name+" "+d['chrom']+" "+str(d['chrom_pos'])+"\n")
        linecache.clearcache()
    
    #Saving the updated dictionary
    !mkdir -p ext_features_dicts/pfam-v32/$domain_name
    
    with open(curr_dir[0]+"/ext_features_dicts/pfam-v"+pfam_version+"/"+domain_name+"/"+domain_name+"_hmm_states_dict_"+today.strftime('%m.%d.%y')+".pik", 'wb') as handle:
        pickle.dump(states_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print("Finished domain "+domain_name)
    linecache.clearcache()
    
missing.close()

Finished domain VGLL4
Finished domain VGPC1_C
Finished domain VHL
Finished domain VHL_C
Finished domain VHP
Finished domain VHS
Finished domain VIGSSK
Finished domain VIR_N
Finished domain VIT
Finished domain VIT_2
Finished domain VKG_Carbox
Finished domain VKOR
Finished domain VMA21
Finished domain VOMI
Finished domain VPS11_C
Finished domain VPS13
Finished domain VPS13_C
Finished domain VPS13_mid_rpt
Finished domain VPS28
Finished domain VPS9
Finished domain VRR_NUC
Finished domain VWA
Finished domain VWA_2
Finished domain VWA_3
Finished domain VWA_N
Finished domain VWA_N2
Finished domain VWC
Finished domain VWD
Finished domain V_ATPase_I
Finished domain Vac14_Fab1_bd
Finished domain Vac14_Fig4_bd
Finished domain Vac_ImportDeg
Finished domain Vasculin
Finished domain Vasohibin
Finished domain Vault
Finished domain Vault_2
Finished domain Vault_3
Finished domain Vault_4
Finished domain Vert_HS_TF
Finished domain Vert_IL3-reg_TF
Finished domain Vezatin
Finished domain Vg_Tdu
Finished d

Finished domain zf-DBF
Finished domain zf-DNA_Pol
Finished domain zf-DNL
Finished domain zf-Di19
Finished domain zf-FCS
Finished domain zf-FPG_IleRS
Finished domain zf-GRF
Finished domain zf-H2C2_2
Finished domain zf-H2C2_5
Finished domain zf-HC5HC2H
Finished domain zf-HC5HC2H_2
Finished domain zf-HIT
Finished domain zf-LITAF-like
Finished domain zf-LYAR
Finished domain zf-MIZ
Finished domain zf-MYND
Finished domain zf-MYST
Finished domain zf-NADH-PPase
Finished domain zf-NF-X1
Finished domain zf-NOSIP
Finished domain zf-NPL4
Finished domain zf-Nse
Finished domain zf-PARP
Finished domain zf-RAG1
Finished domain zf-RING-like
Finished domain zf-RING_10
Finished domain zf-RING_11
Finished domain zf-RING_12
Finished domain zf-RING_14
Finished domain zf-RING_2
Finished domain zf-RING_4
Finished domain zf-RING_5
Finished domain zf-RING_6
Finished domain zf-RING_9
Finished domain zf-RING_UBOX
Finished domain zf-RNPHF
Finished domain zf-RRN7
Finished domain zf-RanBP
Finished domain zf-SAP30
Fi